In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

plt.rcParams['font.family'] = 'NanumGothic'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 주력산업

#### 주력산업 데이터불러오기

In [4]:
major_m = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317상장주력_minmax.csv', index_col=0)
major_s = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317상장주력_stand.csv', index_col=0)

#### 1. LASSO

In [24]:
# Lasso
from sklearn.linear_model import Lasso

def lasso(feature, label):
  lasso = Lasso(alpha = 0.005)
  lasso.fit(feature, label)

  feature_list = lasso.feature_names_in_[lasso.coef_ != 0]
  return feature_list

In [15]:
drop_column = ['회사명', '회계년도', '부실', '설립일','상장일','상장폐지일', '상장협 산업분류(소분류)_x','상장협 산업분류(중분류)_x','회계년','Unnamed: 0']

feature_m = major_m.drop(drop_column, axis=1)
label_m =  major_m['부실']

feature_s = major_s.drop(drop_column, axis=1)
label_s =  major_s['부실']

In [25]:
lasso(feature_m, label_m)

array(['총자본순이익률', '유보액대비율', '매출액(수익)'], dtype=object)

In [26]:
lasso(feature_s, label_s)

array(['매출액순이익률', '자기자본정상영업이익률', '자기자본순이익률', '자본금순이익률', '기타손익비률',
       '금융비용부담률', '적립금비율(재정비율)', '재고자산 대 유동자산비율', '유동부채비율', '비유동부채비율',
       'CASH FLOW 대 매출액비율', '매입채무회전기간', '매출채권회전기간', '소비자물가지수'],
      dtype=object)

#### 2. t-test

##### 상장주력산업 minmax t-test

In [27]:
# 데이터프레임에서 데이터 타입이 object인 컬럼 찾기
object_columns = major_m.select_dtypes(include=['object']).columns

# object 타입인 컬럼 출력
print("데이터 타입이 object인 컬럼:")
for column in object_columns:
    print(column)

데이터 타입이 object인 컬럼:
회사명
회계년도
설립일
상장일
상장폐지일
상장협 산업분류(소분류)_x
상장협 산업분류(중분류)_x


In [28]:
major_m.drop(['회사명',
'회계년도',
'설립일',
'상장일',
'상장폐지일',
'상장협 산업분류(소분류)_x',
'상장협 산업분류(중분류)_x',
'Unnamed: 0',
'회계년'], axis=1, inplace=True)

In [29]:
major_s.drop(['회사명',
'회계년도',
'설립일',
'상장일',
'상장폐지일',
'상장협 산업분류(소분류)_x',
'상장협 산업분류(중분류)_x',
'Unnamed: 0',
'회계년'], axis=1, inplace=True)

In [30]:
# 부도기업과 정상기업의 피처별 등분산비교 (정규성 가정으로 bartlett)
Bad = major_m[major_m['부실']== 1] #Existing Customer
Good = major_m[major_m['부실']== 0] #Attrited Customer

list = []
for i in major_m:
    # lresult = stats.levene(close[i], normal[i])
    lresult = stats.bartlett(Bad[i], Good[i])
    list.append([i,lresult[-1]])

list= pd.DataFrame(list)
list.columns=["피처값",'F-test']
list

,피처값,F-test
0,매출액총이익률,2.804425e-06
1,매출액정상영업이익률,1.037190e-64
2,매출액순이익률,6.885527e-133
3,총자본사업이익률,5.689238e-07
4,총자본정상영업이익률,1.034871e-06
...,...,...
139,수입물량지수,8.612349e-02
140,수입금액지수,2.766927e-01
141,수출물량지수,8.227214e-01
142,수출금액지수,6.191055e-02


In [31]:
# F-test 결과 0.05 이상이면 homo 0.05 이하이면 hetero
list["분산"] =''
list["T-test"] =""
for i in list.index:
    if list.loc[i,"F-test"]>=0.05:
        list.loc[i,"분산"] = "homo"
    else:
        list.loc[i,"분산"] = "hetero"
list

,피처값,F-test,분산,T-test
0,매출액총이익률,2.804425e-06,hetero,
1,매출액정상영업이익률,1.037190e-64,hetero,
2,매출액순이익률,6.885527e-133,hetero,
3,총자본사업이익률,5.689238e-07,hetero,
4,총자본정상영업이익률,1.034871e-06,hetero,
...,...,...,...,...
139,수입물량지수,8.612349e-02,homo,
140,수입금액지수,2.766927e-01,homo,
141,수출물량지수,8.227214e-01,homo,
142,수출금액지수,6.191055e-02,homo,


In [32]:
list[list["분산"]=='homo']

,피처값,F-test,분산,T-test
27,이자부담률,0.758958,homo,
37,재고자산 대 유동자산비율,0.230415,homo,
71,총자본회전률,0.375389,homo,
72,경영자본회전률,0.085533,homo,
78,유동자산회전률,0.210010,homo,
79,당좌자산회전률,0.788357,homo,
86,유형자산회전율,0.587791,homo,
101,유동부채,0.358082,homo,
104,매출액(수익),0.108769,homo,
117,원화의 대미달러,0.303132,homo,


In [33]:
# homo 인 feature 는 student t-test, hetero이면 Welchs T-Test 진행
list["분산"] =''
list["T-test"] =""
for i in list.index:
    if list.loc[i,"F-test"]>=0.05:
        list.loc[i,"분산"] = "homo"
        result = stats.ttest_ind(Bad[list.loc[i,"피처값"]], Good[list.loc[i,"피처값"]], equal_var=True)       ## equal_var = True Student T-test
        list.loc[i,"T-test"] = result[-1]
    else:
        list.loc[i,"분산"] = "hetero"
        result = stats.ttest_ind(Bad[list.loc[i,"피처값"]], Good[list.loc[i,"피처값"]], equal_var=False)      ## equal_var = False Welchs T-Test
        list.loc[i,"T-test"] = result[-1]

list

,피처값,F-test,분산,T-test
0,매출액총이익률,2.804425e-06,hetero,0.000001
1,매출액정상영업이익률,1.037190e-64,hetero,0.0
2,매출액순이익률,6.885527e-133,hetero,0.0
3,총자본사업이익률,5.689238e-07,hetero,0.0
4,총자본정상영업이익률,1.034871e-06,hetero,0.0
...,...,...,...,...
139,수입물량지수,8.612349e-02,homo,0.000195
140,수입금액지수,2.766927e-01,homo,0.006659
141,수출물량지수,8.227214e-01,homo,0.000116
142,수출금액지수,6.191055e-02,homo,0.413193


In [34]:
# 0.05 이상이면 유의미하지 않으므로 0.05 이하인 것만 추출
result = list[list["T-test"]<= 0.05]
result.sort_values('T-test',ascending=False)["피처값"].unique()

array(['비유동자산구성비율', '노동소득분배율', '총포괄이익증가율', '유연탄가격', '순이익증가율', '천연가스가격',
       '알루미늄가격', '수입금액지수', '인건비 대 총비용비율', '금가격', '정상영업이익증가율',
       '한국실업률(계절변동조정)', '아연가격', '유동자산증가율', '지급이자율', '생산자물가지수', '유동부채',
       '원유_WTI', '원유- Dubai', '구리가격', '수출물가지수총지수 (2015=100)', '이윤분배율',
       '원유- Brent', '매출액증가율', '수입물가지수총지수 (2015=100)', '철광석가격', '한국기준금리',
       '국고채(3년)시장금리', '국고채(5년)시장금리', '수입물량지수', '국고채(10년)시장금리', '매출채권회전기간',
       '니켈가격', '자본분배율', '수출물량지수', 'KOSDAQ_종가', 'KOSPI_종가', '소비자물가지수',
       '매출채권 대 매입채무비율', '비유동비율', '재고자산 대 유동자산비율', '재고자산회전률', '현금비율',
       '차입금평균이자율', '유동자산집중도', '투자집중도', '부채비율', '비유동자산집중도', '차입금비율',
       '비유동장기적합률', '매출액총이익률', '유동부채비율', '총자본증가율', '당좌자산회전률', '유동자산회전률',
       '당좌비율', '매입채무회전기간', '매출원가 대 매출액비율', '세금과공과 대 세금과공과 차감전순이익률',
       '비유동자산회전률', '자기자본증가율', '이자부담률', '기타손익비률', '금융손익비율', '유동비율',
       '금융비용 대 총비용비율', '경영자본회전률', '적립금비율(재정비율)', '금융비용부담률', '총자본회전률',
       '부가가치율', 'CASH FLOW 대 매출액비율', '타인자본구성비율', '자기자본구성비율',
       '비유동부채 대 순운전자본비율', '매출액순

##### 상장주력산업 minmax t-test 결과

In [35]:
# major_m에서의 t-test결과
result.sort_values('T-test',ascending=True).dropna()[:20]

,피처값,F-test,분산,T-test
62,유보액 대 납입자본배율,4.890330e-44,hetero,0.0
43,자기자본배율,3.969223e-42,hetero,0.0
30,유보율,5.050583e-41,hetero,0.0
32,1주당매출액,1.887231e-64,hetero,0.0
35,1주당정상영업이익,2.225081e-53,hetero,0.0
90,부가가치(백만원),1.611778e-90,hetero,0.0
103,자본,4.044891e-96,hetero,0.0
34,1주당 CASH FLOW,6.220944e-62,hetero,0.0
98,영업활동으로 인한 현금흐름(간접법),4.991764e-98,hetero,0.0
67,CASH FLOW 대 차입금비율,4.739854e-114,hetero,0.0


##### 상장주력산업 standard t-test

In [36]:
# 부도기업과 정상기업의 피처별 등분산비교 (정규성 가정으로 bartlett)
Bad = major_s[major_s['부실']== 1] #Existing Customer
Good = major_s[major_s['부실']== 0] #Attrited Customer

list = []
for i in major_s:
    # lresult = stats.levene(close[i], normal[i])
    lresult = stats.bartlett(Bad[i], Good[i])
    list.append([i,lresult[-1]])

list= pd.DataFrame(list)
list.columns=["피처값",'F-test']
list

,피처값,F-test
0,매출액총이익률,2.804425e-06
1,매출액정상영업이익률,1.037190e-64
2,매출액순이익률,6.885527e-133
3,총자본사업이익률,5.689238e-07
4,총자본정상영업이익률,1.034871e-06
...,...,...
139,수입물량지수,8.612349e-02
140,수입금액지수,2.766927e-01
141,수출물량지수,8.227214e-01
142,수출금액지수,6.191055e-02


In [37]:
# F-test 결과 0.05 이상이면 homo 0.05 이하이면 hetero
list["분산"] =''
list["T-test"] =""
for i in list.index:
    if list.loc[i,"F-test"]>=0.05:
        list.loc[i,"분산"] = "homo"
    else:
        list.loc[i,"분산"] = "hetero"
list

,피처값,F-test,분산,T-test
0,매출액총이익률,2.804425e-06,hetero,
1,매출액정상영업이익률,1.037190e-64,hetero,
2,매출액순이익률,6.885527e-133,hetero,
3,총자본사업이익률,5.689238e-07,hetero,
4,총자본정상영업이익률,1.034871e-06,hetero,
...,...,...,...,...
139,수입물량지수,8.612349e-02,homo,
140,수입금액지수,2.766927e-01,homo,
141,수출물량지수,8.227214e-01,homo,
142,수출금액지수,6.191055e-02,homo,


In [38]:
list[list["분산"]=='homo']

,피처값,F-test,분산,T-test
27,이자부담률,0.758958,homo,
37,재고자산 대 유동자산비율,0.230415,homo,
71,총자본회전률,0.375389,homo,
72,경영자본회전률,0.085533,homo,
78,유동자산회전률,0.210010,homo,
79,당좌자산회전률,0.788357,homo,
86,유형자산회전율,0.587791,homo,
101,유동부채,0.358082,homo,
104,매출액(수익),0.108769,homo,
117,원화의 대미달러,0.303132,homo,


In [39]:
# homo 인 feature 는 student t-test, hetero이면 Welchs T-Test 진행
list["분산"] =''
list["T-test"] =""
for i in list.index:
    if list.loc[i,"F-test"]>=0.05:
        list.loc[i,"분산"] = "homo"
        result = stats.ttest_ind(Bad[list.loc[i,"피처값"]], Good[list.loc[i,"피처값"]], equal_var=True)       ## equal_var = True Student T-test
        list.loc[i,"T-test"] = result[-1]
    else:
        list.loc[i,"분산"] = "hetero"
        result = stats.ttest_ind(Bad[list.loc[i,"피처값"]], Good[list.loc[i,"피처값"]], equal_var=False)      ## equal_var = False Welchs T-Test
        list.loc[i,"T-test"] = result[-1]

list

,피처값,F-test,분산,T-test
0,매출액총이익률,2.804425e-06,hetero,0.000001
1,매출액정상영업이익률,1.037190e-64,hetero,0.0
2,매출액순이익률,6.885527e-133,hetero,0.0
3,총자본사업이익률,5.689238e-07,hetero,0.0
4,총자본정상영업이익률,1.034871e-06,hetero,0.0
...,...,...,...,...
139,수입물량지수,8.612349e-02,homo,0.000195
140,수입금액지수,2.766927e-01,homo,0.006659
141,수출물량지수,8.227214e-01,homo,0.000116
142,수출금액지수,6.191055e-02,homo,0.413193


In [40]:
# 0.05 이하인 것만 추출
result = list[list["T-test"]<= 0.05]
result.sort_values('T-test',ascending=True)["피처값"].unique()

array(['부실', '유보액 대 납입자본배율', '자기자본배율', '유보율', '1주당매출액', '1주당정상영업이익',
       '부가가치(백만원)', '자본', '1주당 CASH FLOW', '영업활동으로 인한 현금흐름(간접법)',
       'CASH FLOW 대 차입금비율', '자본금정상영업이익률', '정상영업이익대비이자보상배율', '총자본정상영업이익률',
       '총자본사업이익률', '1주당순이익', '경영자본정상영업이익률', '자본금순이익률', '순운전자본회전률',
       '유보액대비율', 'CASH FLOW 대 총자본비율', '매출액(수익)', '총자본순이익률', '기업순이익률',
       'CASH FLOW 대 부채비율', '기계투자효율', '타인자본회전률', '경영자본순이익률', '총자본투자효율',
       '자기자본정상영업이익률', '자기자본순이익률', '재고자산 대 순운전자본비율', '매입채무회전률', '자본금회전률',
       '수지비율(관계기업투자손익 제외)', '투자활동으로 인한 현금흐름', '순운전자본비율', '자산_x',
       '매출액정상영업이익률', '영업비용 대 영업수익비율', '설비투자효율', '차입금의존도', '매출액순이익률',
       '비유동부채 대 순운전자본비율', '자기자본구성비율', '타인자본구성비율', 'CASH FLOW 대 매출액비율',
       '부가가치율', '총자본회전률', '금융비용부담률', '적립금비율(재정비율)', '경영자본회전률',
       '금융비용 대 총비용비율', '유동비율', '금융손익비율', '기타손익비률', '이자부담률', '자기자본증가율',
       '비유동자산회전률', '세금과공과 대 세금과공과 차감전순이익률', '매출원가 대 매출액비율', '매입채무회전기간',
       '당좌비율', '유동자산회전률', '당좌자산회전률', '총자본증가율', '유동부채비율', '매출액총이익률',
       '비유동장기적합률', '차입금비율', '비유

##### 상장주력산업 standard t-test 결과

In [ ]:
# major_s에서의 t-test결과
result.sort_values('T-test',ascending=True).dropna()[:20]

,피처값,F-test,분산,T-test
62,유보액 대 납입자본배율,4.890330e-44,hetero,0.0
43,자기자본배율,3.969223e-42,hetero,0.0
30,유보율,5.050583e-41,hetero,0.0
32,1주당매출액,1.887231e-64,hetero,0.0
35,1주당정상영업이익,2.225081e-53,hetero,0.0
90,부가가치(백만원),1.611778e-90,hetero,0.0
103,자본,4.044891e-96,hetero,0.0
34,1주당 CASH FLOW,6.220944e-62,hetero,0.0
98,영업활동으로 인한 현금흐름(간접법),4.991764e-98,hetero,0.0
67,CASH FLOW 대 차입금비율,4.739854e-114,hetero,0.0


#### 3. Logit

##### 주력산업 minmax logit

In [41]:
major_m = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317상장주력_minmax.csv', index_col=0)
major_s = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317상장주력_stand.csv', index_col=0)

In [42]:
# drop해야할 column
drop_column = ['회사명', '회계년도', '설립일','상장일','상장폐지일', '상장협 산업분류(소분류)_x','상장협 산업분류(중분류)_x','회계년','Unnamed: 0']
major_m.drop(drop_column, axis=1, inplace=True)
major_s.drop(drop_column, axis=1, inplace=True)

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
import statsmodels.api as sm

lr_clf = LogisticRegression()

feature_m = major_m.drop('부실',axis=1)
label_m =  major_m['부실']

feature_s = major_s.drop('부실',axis=1)
label_s =  major_s['부실']

In [44]:
logit = SelectFromModel(LogisticRegression())
logit.fit(feature_m, label_m)
logit_support = logit.get_support()
lr_feature = feature_m.loc[:,logit_support].columns.tolist()

##### 결과

In [45]:
# minmax의 logit 결과
lr_feature

['매출액정상영업이익률',
 '매출액순이익률',
 '총자본사업이익률',
 '총자본정상영업이익률',
 '총자본순이익률',
 '자기자본정상영업이익률',
 '자기자본순이익률',
 '자본금정상영업이익률',
 '자본금순이익률',
 '매출원가 대 매출액비율',
 '기타손익비률',
 '금융손익비율',
 '기업순이익률',
 '인건비 대 총비용비율',
 '금융비용 대 총비용비율',
 '감가상각비 대 총비용비율',
 '지급이자율',
 '적립금비율(재정비율)',
 '재고자산 대 유동자산비율',
 '당좌자산구성비율',
 '비유동장기적합률',
 '매출채권 대 매입채무비율',
 '유동부채비율',
 '비유동부채비율',
 '비유동부채 대 순운전자본비율',
 '순운전자본비율',
 '차입금의존도',
 '차입금비율',
 '유보액대비율',
 '투자집중도',
 'CASH FLOW 대 총자본비율',
 'CASH FLOW 대 매출액비율',
 '자본금회전률',
 '타인자본회전률',
 '매입채무회전률',
 '매입채무회전기간',
 '재고자산회전률',
 '재고자산회전기간',
 '매출채권회전률',
 '매출채권회전기간',
 '비유동자산회전률',
 '순운전자본회전률',
 '운전자본회전률',
 '1회전기간',
 '총자본투자효율',
 '노동소득분배율',
 '매출액(수익)',
 '유형자산증가율',
 '유동자산증가율',
 '비유동자산증가율',
 '재고자산증가율',
 '총포괄이익증가율',
 '총자본증가율',
 '원화의 대미달러',
 '아연가격']

##### 주력산업 standard logit

In [46]:
logit = SelectFromModel(LogisticRegression())
logit.fit(feature_s, label_s)
logit_support = logit.get_support()
lr_feature = feature_s.loc[:,logit_support].columns.tolist()

##### 결과

In [47]:
# minmax의 logit 결과
lr_feature

['매출액총이익률',
 '매출액정상영업이익률',
 '총자본사업이익률',
 '총자본순이익률',
 '자기자본정상영업이익률',
 '경영자본정상영업이익률',
 '자본금정상영업이익률',
 '매출원가 대 매출액비율',
 '기타손익비률',
 '금융비용부담률',
 '수지비율(관계기업투자손익 제외)',
 '인건비 대 총비용비율',
 '금융비용 대 총비용비율',
 '감가상각비 대 총비용비율',
 '유보율',
 '1주당매출액',
 '1주당순이익',
 '1주당정상영업이익',
 '재고자산 대 유동자산비율',
 '유동자산 대 비유동자산비율',
 '자기자본배율',
 '비유동비율',
 '매출채권 대 매입채무비율',
 '부채비율',
 '비유동부채비율',
 '비유동부채 대 순운전자본비율',
 '순운전자본비율',
 '차입금의존도',
 '차입금비율',
 '유동자산집중도',
 '비유동자산집중도',
 '투자집중도',
 'CASH FLOW 대 부채비율',
 'CASH FLOW 대 차입금비율',
 'CASH FLOW 대 총자본비율',
 'CASH FLOW 대 매출액비율',
 '경영자본회전률',
 '자본금회전률',
 '타인자본회전률',
 '매입채무회전률',
 '당좌자산회전률',
 '재고자산회전률',
 '매출채권회전기간',
 '비유동자산회전률',
 '순운전자본회전률',
 '노동소득분배율',
 '자본분배율',
 '투자활동으로 인한 현금흐름',
 '자본',
 '매출액(수익)',
 '투자부동산증가율',
 '총자본증가율']

#### 4. SelectKBest

##### 주력산업 minmax SelectKBest

In [50]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X_new = SelectKBest(f_classif, k=20).fit(feature_m, label_m)
X_new.get_feature_names_out()

array(['매출액정상영업이익률', '매출액순이익률', '총자본사업이익률', '총자본정상영업이익률', '총자본순이익률',
       '자기자본정상영업이익률', '자기자본순이익률', '경영자본정상영업이익률', '경영자본순이익률',
       '영업비용 대 영업수익비율', '기타손익비률', '금융비용부담률', '기업순이익률',
       '수지비율(관계기업투자손익 제외)', '적립금비율(재정비율)', '유동부채비율', 'CASH FLOW 대 총자본비율',
       'CASH FLOW 대 매출액비율', '총자본투자효율', '부가가치율'], dtype=object)

##### 주력산업 standard SelectKBest

In [51]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X_new = SelectKBest(f_classif, k=20).fit(feature_s, label_s)
X_new.get_feature_names_out()

array(['매출액정상영업이익률', '매출액순이익률', '총자본사업이익률', '총자본정상영업이익률', '총자본순이익률',
       '자기자본정상영업이익률', '자기자본순이익률', '경영자본정상영업이익률', '경영자본순이익률',
       '영업비용 대 영업수익비율', '기타손익비률', '금융비용부담률', '기업순이익률',
       '수지비율(관계기업투자손익 제외)', '적립금비율(재정비율)', '유동부채비율', 'CASH FLOW 대 총자본비율',
       'CASH FLOW 대 매출액비율', '총자본투자효율', '부가가치율'], dtype=object)

## 비주력산업

In [112]:
minor_m = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317상장비주력_minmax.csv', index_col=0)
minor_s = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317상장비주력_stand.csv', index_col=0)

#### 1. LASSO

##### Lasso 불러오기

In [113]:
# Lasso
from sklearn.linear_model import Lasso

def lasso(feature, label):
  lasso = Lasso(alpha = 0.005)
  lasso.fit(feature, label)

  feature_list = lasso.feature_names_in_[lasso.coef_ != 0]
  return feature_list

In [114]:
drop_column = ['회사명', '회계년도', '부실', '설립일','상장일','상장폐지일', '상장협 산업분류(소분류)_x','상장협 산업분류(중분류)_x']

feature_m = minor_m.drop(drop_column, axis=1)
label_m =  minor_m['부실']

feature_s = minor_s.drop(drop_column, axis=1)
label_s =  minor_s['부실']

##### 비주력산업 minmax lasso 결과

In [115]:
lasso(feature_m, label_m)

array(['매출액(수익)'], dtype=object)

##### 비주력산업 standard lasso 결과

In [116]:
lasso(feature_s, label_s)

array(['자기자본순이익률', '경영자본순이익률', '금융비용부담률', '지급이자율', '유보액대비율', '매입채무회전기간',
       '당좌자산회전률', '매출채권회전률', '자본분배율', '유연탄가격', '금가격', '수출물량지수'],
      dtype=object)

#### 2. t-test

##### 상장비주력산업 minmax t-test

In [86]:
# 데이터프레임에서 데이터 타입이 object인 컬럼 찾기
object_columns = minor_m.select_dtypes(include=['object']).columns

# object 타입인 컬럼 출력
print("데이터 타입이 object인 컬럼:")
for column in object_columns:
    print(column)

데이터 타입이 object인 컬럼:
회사명
회계년도
설립일
상장일
상장폐지일
상장협 산업분류(소분류)_x
상장협 산업분류(중분류)_x


In [87]:
minor_m.drop(['회사명',
'회계년도',
'설립일',
'상장일',
'상장폐지일',
'상장협 산업분류(소분류)_x',
'상장협 산업분류(중분류)_x',
], axis=1, inplace=True)

In [88]:
minor_s.drop(['회사명',
'회계년도',
'설립일',
'상장일',
'상장폐지일',
'상장협 산업분류(소분류)_x',
'상장협 산업분류(중분류)_x'], axis=1, inplace=True)

In [89]:
# 부도기업과 정상기업의 피처별 등분산비교 (정규성 가정으로 bartlett)
Bad = minor_m[minor_m['부실']== 1] #Existing Customer
Good = minor_m[minor_m['부실']== 0] #Attrited Customer

list = []
for i in minor_m:
    # lresult = stats.levene(close[i], normal[i])
    lresult = stats.bartlett(Bad[i], Good[i])
    list.append([i,lresult[-1]])

list= pd.DataFrame(list)
list.columns=["피처값",'F-test']
list

,피처값,F-test
0,매출액총이익률,6.351505e-04
1,매출액정상영업이익률,1.189466e-13
2,매출액순이익률,1.344971e-22
3,총자본사업이익률,1.502189e-08
4,총자본정상영업이익률,5.266946e-09
...,...,...
139,수입물량지수,7.216133e-01
140,수입금액지수,1.115285e-01
141,수출물량지수,6.076679e-01
142,수출금액지수,2.631237e-02


In [90]:
# F-test 결과 0.05 이상이면 homo 0.05 이하이면 hetero
list["분산"] =''
list["T-test"] =""
for i in list.index:
    if list.loc[i,"F-test"]>=0.05:
        list.loc[i,"분산"] = "homo"
    else:
        list.loc[i,"분산"] = "hetero"
list

,피처값,F-test,분산,T-test
0,매출액총이익률,6.351505e-04,hetero,
1,매출액정상영업이익률,1.189466e-13,hetero,
2,매출액순이익률,1.344971e-22,hetero,
3,총자본사업이익률,1.502189e-08,hetero,
4,총자본정상영업이익률,5.266946e-09,hetero,
...,...,...,...,...
139,수입물량지수,7.216133e-01,homo,
140,수입금액지수,1.115285e-01,homo,
141,수출물량지수,6.076679e-01,homo,
142,수출금액지수,2.631237e-02,hetero,


In [91]:
list[list["분산"]=='homo']

,피처값,F-test,분산,T-test
12,매출원가 대 매출액비율,0.927358,homo,
22,인건비 대 총비용비율,0.335653,homo,
23,금융비용 대 총비용비율,0.074983,homo,
33,1주당순이익,0.238192,homo,
36,유동자산구성비율,0.139772,homo,
37,재고자산 대 유동자산비율,0.340703,homo,
39,당좌자산구성비율,0.114841,homo,
40,비유동자산구성비율,0.140913,homo,
46,유동비율,0.198476,homo,
47,당좌비율,0.123960,homo,


In [92]:
# homo 인 feature 는 student t-test, hetero이면 Welchs T-Test 진행
list["분산"] =''
list["T-test"] =""
for i in list.index:
    if list.loc[i,"F-test"]>=0.05:
        list.loc[i,"분산"] = "homo"
        result = stats.ttest_ind(Bad[list.loc[i,"피처값"]], Good[list.loc[i,"피처값"]], equal_var=True)       ## equal_var = True Student T-test
        list.loc[i,"T-test"] = result[-1]
    else:
        list.loc[i,"분산"] = "hetero"
        result = stats.ttest_ind(Bad[list.loc[i,"피처값"]], Good[list.loc[i,"피처값"]], equal_var=False)      ## equal_var = False Welchs T-Test
        list.loc[i,"T-test"] = result[-1]

list

,피처값,F-test,분산,T-test
0,매출액총이익률,6.351505e-04,hetero,0.06402
1,매출액정상영업이익률,1.189466e-13,hetero,0.0
2,매출액순이익률,1.344971e-22,hetero,0.0
3,총자본사업이익률,1.502189e-08,hetero,0.0
4,총자본정상영업이익률,5.266946e-09,hetero,0.0
...,...,...,...,...
139,수입물량지수,7.216133e-01,homo,0.000139
140,수입금액지수,1.115285e-01,homo,0.004014
141,수출물량지수,6.076679e-01,homo,0.000013
142,수출금액지수,2.631237e-02,hetero,0.657865


In [93]:
# 0.05 이상이면 유의미하지 않으므로 0.05 이하인 것만 추출
result = list[list["T-test"]<= 0.05]
result.sort_values('T-test',ascending=False)["피처값"].unique()
# result.sort_values('T-test',ascending=False)["피처값"]

array(['자기자본회전률', '총포괄이익증가율', 'CASH FLOW 대 차입금비율', 'KOSPI_종가', '차입금비율',
       '유동비율', '이윤분배율', '현금비율', '부채비율', '매출채권 대 매입채무비율', '투자집중도',
       '비유동자산집중도', '자기자본증가율', '유동부채비율', '자본분배율', '한국실업률(계절변동조정)', '지급이자율',
       '정상영업이익증가율', '감가상각비 대 총비용비율', '수입금액지수', '유동자산증가율', '차입금평균이자율',
       '유동자산집중도', '매입채무회전기간', 'KOSDAQ_종가', '원유_WTI', '유동부채',
       '재고자산 대 유동자산비율', '매출원가 대 매출액비율', '재공품회전률', '원유- Dubai', '비유동자산증가율',
       '원유- Brent', '생산자물가지수', '수입물량지수', '국고채(10년)시장금리', '금융손익비율',
       '국고채(5년)시장금리', '알루미늄가격', 'CASH FLOW 대 매출액비율', '철광석가격', '니켈가격',
       '총자본증가율', '소비자물가지수', '수입물가지수총지수 (2015=100)', '구리가격', '국고채(3년)시장금리',
       '유연탄가격', '수출물량지수', '한국기준금리', '수출물가지수총지수 (2015=100)', '적립금비율(재정비율)',
       '금가격', 'CASH FLOW 대 총자본비율', '기계투자효율', '기타손익비률', '금융비용부담률',
       '타인자본구성비율', '경영자본회전률', '설비투자효율', '자기자본구성비율', '자기자본정상영업이익률',
       '1주당 CASH FLOW', '자기자본순이익률', '1주당순이익', '부가가치율', '차입금의존도',
       '매출액정상영업이익률', '영업비용 대 영업수익비율', '비유동자산회전률', '자산_x', '매출액순이익률',
       '순운전자본비율', 'CASH FLO

##### 상장비주력산업 minmax t-test 결과

In [117]:
# minor_m에서의 t-test결과
result.sort_values('T-test',ascending=True).dropna()[:20]

,피처값,F-test,분산,T-test
62,유보액 대 납입자본배율,1.243706e-54,hetero,0.0
30,유보율,2.761137e-51,hetero,0.0
43,자기자본배율,1.702161e-49,hetero,0.0
103,자본,6.132845e-78,hetero,0.0
90,부가가치(백만원),3.643431e-55,hetero,0.0
74,자본금회전률,2.845133e-39,hetero,0.0
61,유보액대비율,5.781458e-01,homo,0.0
32,1주당매출액,3.112727e-37,hetero,0.0
91,총자본투자효율,1.540805e-01,homo,0.0
75,타인자본회전률,1.244573e-17,hetero,0.0


##### 상장비주력산업 standard t-test

In [95]:
# 부도기업과 정상기업의 피처별 등분산비교 (정규성 가정으로 bartlett)
Bad = minor_s[minor_s['부실']== 1] #Existing Customer
Good = minor_s[minor_s['부실']== 0] #Attrited Customer

list = []
for i in minor_s:
    # lresult = stats.levene(close[i], normal[i])
    lresult = stats.bartlett(Bad[i], Good[i])
    list.append([i,lresult[-1]])

list= pd.DataFrame(list)
list.columns=["피처값",'F-test']
list

,피처값,F-test
0,매출액총이익률,6.351505e-04
1,매출액정상영업이익률,1.189466e-13
2,매출액순이익률,1.344971e-22
3,총자본사업이익률,1.502189e-08
4,총자본정상영업이익률,5.266946e-09
...,...,...
139,수입물량지수,7.216133e-01
140,수입금액지수,1.115285e-01
141,수출물량지수,6.076679e-01
142,수출금액지수,2.631237e-02


In [96]:
# F-test 결과 0.05 이상이면 homo 0.05 이하이면 hetero
list["분산"] =''
list["T-test"] =""
for i in list.index:
    if list.loc[i,"F-test"]>=0.05:
        list.loc[i,"분산"] = "homo"
    else:
        list.loc[i,"분산"] = "hetero"
list

,피처값,F-test,분산,T-test
0,매출액총이익률,6.351505e-04,hetero,
1,매출액정상영업이익률,1.189466e-13,hetero,
2,매출액순이익률,1.344971e-22,hetero,
3,총자본사업이익률,1.502189e-08,hetero,
4,총자본정상영업이익률,5.266946e-09,hetero,
...,...,...,...,...
139,수입물량지수,7.216133e-01,homo,
140,수입금액지수,1.115285e-01,homo,
141,수출물량지수,6.076679e-01,homo,
142,수출금액지수,2.631237e-02,hetero,


In [97]:
list[list["분산"]=='homo']

,피처값,F-test,분산,T-test
12,매출원가 대 매출액비율,0.927358,homo,
22,인건비 대 총비용비율,0.335653,homo,
23,금융비용 대 총비용비율,0.074983,homo,
33,1주당순이익,0.238192,homo,
36,유동자산구성비율,0.139772,homo,
37,재고자산 대 유동자산비율,0.340703,homo,
39,당좌자산구성비율,0.114841,homo,
40,비유동자산구성비율,0.140913,homo,
46,유동비율,0.198476,homo,
47,당좌비율,0.123960,homo,


In [98]:
# homo 인 feature 는 student t-test, hetero이면 Welchs T-Test 진행
list["분산"] =''
list["T-test"] =""
for i in list.index:
    if list.loc[i,"F-test"]>=0.05:
        list.loc[i,"분산"] = "homo"
        result = stats.ttest_ind(Bad[list.loc[i,"피처값"]], Good[list.loc[i,"피처값"]], equal_var=True)       ## equal_var = True Student T-test
        list.loc[i,"T-test"] = result[-1]
    else:
        list.loc[i,"분산"] = "hetero"
        result = stats.ttest_ind(Bad[list.loc[i,"피처값"]], Good[list.loc[i,"피처값"]], equal_var=False)      ## equal_var = False Welchs T-Test
        list.loc[i,"T-test"] = result[-1]

list

,피처값,F-test,분산,T-test
0,매출액총이익률,6.351505e-04,hetero,0.06402
1,매출액정상영업이익률,1.189466e-13,hetero,0.0
2,매출액순이익률,1.344971e-22,hetero,0.0
3,총자본사업이익률,1.502189e-08,hetero,0.0
4,총자본정상영업이익률,5.266946e-09,hetero,0.0
...,...,...,...,...
139,수입물량지수,7.216133e-01,homo,0.000139
140,수입금액지수,1.115285e-01,homo,0.004014
141,수출물량지수,6.076679e-01,homo,0.000013
142,수출금액지수,2.631237e-02,hetero,0.657865


In [99]:
# 0.05 이상이면 유의미하지 않으므로 0.05 이하인 것만 추출
result = list[list["T-test"]<= 0.05]
result.sort_values('T-test',ascending=False)["피처값"].unique()
# result.sort_values('T-test',ascending=False)["피처값"]

array(['자기자본회전률', '총포괄이익증가율', 'CASH FLOW 대 차입금비율', 'KOSPI_종가', '차입금비율',
       '유동비율', '이윤분배율', '현금비율', '부채비율', '매출채권 대 매입채무비율', '투자집중도',
       '비유동자산집중도', '자기자본증가율', '유동부채비율', '자본분배율', '한국실업률(계절변동조정)', '지급이자율',
       '정상영업이익증가율', '감가상각비 대 총비용비율', '수입금액지수', '유동자산증가율', '차입금평균이자율',
       '유동자산집중도', '매입채무회전기간', 'KOSDAQ_종가', '원유_WTI', '유동부채',
       '재고자산 대 유동자산비율', '매출원가 대 매출액비율', '재공품회전률', '원유- Dubai', '비유동자산증가율',
       '원유- Brent', '생산자물가지수', '수입물량지수', '국고채(10년)시장금리', '금융손익비율',
       '국고채(5년)시장금리', '알루미늄가격', 'CASH FLOW 대 매출액비율', '철광석가격', '니켈가격',
       '총자본증가율', '소비자물가지수', '수입물가지수총지수 (2015=100)', '구리가격', '국고채(3년)시장금리',
       '유연탄가격', '수출물량지수', '한국기준금리', '수출물가지수총지수 (2015=100)', '적립금비율(재정비율)',
       '금가격', 'CASH FLOW 대 총자본비율', '기계투자효율', '기타손익비률', '금융비용부담률',
       '타인자본구성비율', '경영자본회전률', '설비투자효율', '자기자본구성비율', '자기자본정상영업이익률',
       '1주당 CASH FLOW', '자기자본순이익률', '1주당순이익', '부가가치율', '차입금의존도',
       '매출액정상영업이익률', '영업비용 대 영업수익비율', '비유동자산회전률', '자산_x', '매출액순이익률',
       '순운전자본비율', 'CASH FLO

##### 상장비주력산업 standard t-test 결과

In [118]:
# minor_m에서의 t-test결과
result.sort_values('T-test',ascending=True).dropna()[:20]

,피처값,F-test,분산,T-test
62,유보액 대 납입자본배율,1.243706e-54,hetero,0.0
30,유보율,2.761137e-51,hetero,0.0
43,자기자본배율,1.702161e-49,hetero,0.0
103,자본,6.132845e-78,hetero,0.0
90,부가가치(백만원),3.643431e-55,hetero,0.0
74,자본금회전률,2.845133e-39,hetero,0.0
61,유보액대비율,5.781458e-01,homo,0.0
32,1주당매출액,3.112727e-37,hetero,0.0
91,총자본투자효율,1.540805e-01,homo,0.0
75,타인자본회전률,1.244573e-17,hetero,0.0


#### 3. Logit

##### 비주력산업 minmax logit

In [101]:
minor_m = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317상장비주력_minmax.csv', index_col=0)
minor_s = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317상장비주력_stand.csv', index_col=0)

In [102]:
# drop해야할 column
drop_column = ['회사명', '회계년도', '설립일','상장일','상장폐지일', '상장협 산업분류(소분류)_x','상장협 산업분류(중분류)_x']
minor_m.drop(drop_column, axis=1, inplace=True)
minor_s.drop(drop_column, axis=1, inplace=True)

In [103]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
import statsmodels.api as sm

lr_clf = LogisticRegression()

feature_m = minor_m.drop('부실',axis=1)
label_m =  minor_m['부실']

feature_s = minor_s.drop('부실',axis=1)
label_s =  minor_s['부실']

In [104]:
logit = SelectFromModel(LogisticRegression())
logit.fit(feature_m, label_m)
logit_support = logit.get_support()
lr_feature = feature_m.loc[:,logit_support].columns.tolist()

##### 결과

In [105]:
# minmax의 logit 결과
lr_feature

['매출액총이익률',
 '매출액정상영업이익률',
 '총자본사업이익률',
 '총자본정상영업이익률',
 '총자본순이익률',
 '경영자본정상영업이익률',
 '경영자본순이익률',
 '매출원가 대 매출액비율',
 '금융손익비율',
 '금융비용부담률',
 '기업순이익률',
 '수지비율(관계기업투자손익 제외)',
 '금융비용 대 총비용비율',
 '감가상각비 대 총비용비율',
 '감가상각률',
 '이자부담률',
 '지급이자율',
 '차입금평균이자율',
 '재고자산 대 유동자산비율',
 '유동자산 대 비유동자산비율',
 '비유동자산구성비율',
 '자기자본구성비율',
 '타인자본구성비율',
 '비유동비율',
 '비유동장기적합률',
 '현금비율',
 '재고자산 대 순운전자본비율',
 '매입채무 대 재고자산비율',
 '순운전자본비율',
 '차입금의존도',
 '유보액대비율',
 '비유동자산집중도',
 'CASH FLOW 대 총자본비율',
 '총자본회전률',
 '타인자본회전률',
 '매입채무회전기간',
 '유동자산회전률',
 '당좌자산회전률',
 '매출채권회전률',
 '매출채권회전기간',
 '비유동자산회전률',
 '순운전자본회전률',
 '운전자본회전률',
 '부가가치율',
 '투자활동으로 인한 현금흐름',
 '유동부채',
 '자산_x',
 '매출액(수익)',
 '유동자산증가율',
 '비유동자산증가율',
 '정상영업이익증가율',
 '원화의 대미달러']

##### 주력산업 standard logit

In [106]:
logit = SelectFromModel(LogisticRegression())
logit.fit(feature_s, label_s)
logit_support = logit.get_support()
lr_feature = feature_s.loc[:,logit_support].columns.tolist()

##### 결과

In [107]:
# minmax의 logit 결과
lr_feature

['매출액총이익률',
 '자기자본정상영업이익률',
 '경영자본정상영업이익률',
 '경영자본순이익률',
 '자본금순이익률',
 '매출원가 대 매출액비율',
 '기타손익비률',
 '금융손익비율',
 '인건비 대 총비용비율',
 '금융비용 대 총비용비율',
 '감가상각비 대 총비용비율',
 '감가상각률',
 '누적감가상각률',
 '이자부담률',
 '지급이자율',
 '차입금평균이자율',
 '유보율',
 '재고자산 대 유동자산비율',
 '유동자산 대 비유동자산비율',
 '자기자본배율',
 '비유동비율',
 '현금비율',
 '매출채권비율',
 '매출채권 대 매입채무비율',
 '유동부채비율',
 '비유동부채비율',
 '차입금의존도',
 '유보액대비율',
 '유보액 대 납입자본배율',
 '유동자산집중도',
 '비유동자산집중도',
 '투자집중도',
 'CASH FLOW 대 부채비율',
 'CASH FLOW 대 매출액비율',
 '총자본회전률',
 '경영자본회전률',
 '자본금회전률',
 '타인자본회전률',
 '매입채무회전률',
 '매입채무회전기간',
 '유동자산회전률',
 '당좌자산회전률',
 '재고자산회전률',
 '재공품회전률',
 '매출채권회전률',
 '매출채권회전기간',
 '비유동자산회전률',
 '순운전자본회전률',
 '운전자본회전률',
 '총자본투자효율',
 '부가가치율',
 '노동소득분배율',
 '자본분배율',
 '자산_x',
 '자본',
 '유동자산증가율',
 '비유동자산증가율',
 '총자본증가율',
 '유연탄가격']

#### 4. SelectKBest

##### 비주력산업 minmax SelectKBest

In [119]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X_new = SelectKBest(f_classif, k=20).fit(feature_m, label_m)
X_new.get_feature_names_out()

array(['매출액정상영업이익률', '매출액순이익률', '총자본사업이익률', '총자본정상영업이익률', '총자본순이익률',
       '자기자본정상영업이익률', '자기자본순이익률', '경영자본정상영업이익률', '경영자본순이익률',
       '영업비용 대 영업수익비율', '기타손익비률', '금융비용부담률', '기업순이익률',
       '수지비율(관계기업투자손익 제외)', '지급이자율', '적립금비율(재정비율)', '유보액대비율',
       'CASH FLOW 대 총자본비율', '총자본투자효율', '부가가치율'], dtype=object)

##### 비주력산업 standard SelectKBest

In [120]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X_new = SelectKBest(f_classif, k=20).fit(feature_s, label_s)
X_new.get_feature_names_out()

array(['매출액정상영업이익률', '매출액순이익률', '총자본사업이익률', '총자본정상영업이익률', '총자본순이익률',
       '자기자본정상영업이익률', '자기자본순이익률', '경영자본정상영업이익률', '경영자본순이익률',
       '영업비용 대 영업수익비율', '기타손익비률', '금융비용부담률', '기업순이익률',
       '수지비율(관계기업투자손익 제외)', '지급이자율', '적립금비율(재정비율)', '유보액대비율',
       'CASH FLOW 대 총자본비율', '총자본투자효율', '부가가치율'], dtype=object)